In [ ]:
pip install annoy

     |████████████████████████████████| 646 kB 5.4 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391680 sha256=6eebf791ff0007243d71200afc8c2d93bf646ed7bdc78a24439fa41608d327bd
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [ ]:
pip install pymorphy2

     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 8.2 MB/s 


In [ ]:
pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=32e134e4bf6b5ade480a2f2f0987193e01c4d8639b8844aced048a0b10983ccd
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [ ]:
pip install StringIO

ERROR: Could not find a version that satisfies the requirement StringIO (from versions: none)
ERROR: No matching distribution found for StringIO


In [1]:
from tqdm.notebook import tqdm
import os
import string
import annoy
import io

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec
from io import StringIO

import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: ignored

In [ ]:
tqdm.pandas()  # enables ProcessBar function


In [ ]:
import warnings  
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['ProductsDataset.csv']))

In [ ]:
df.head(5)

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...


Предобработка текста

In [ ]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)


def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
import time

t = time.perf_counter()
sentences = []
c = 0

for line in tqdm(df):
    spls = preprocess_txt(line)
    sentences.append(spls)
    c += 1
    if c > 500000:
        break

print('Elapsed, s:', time.perf_counter() - t)

  0%|          | 0/35548 [00:00<?, ?it/s]

Elapsed, s: 0.10756267100077821


In [ ]:
# Обучим модель word2vec на наших вопросах
sentences = [i for i in sentences if len(i) > 2]
model = Word2Vec(sentences=sentences, size=100, min_count=1, window=5)
model.save("w2v_model")
print('Elapsed, s:', time.perf_counter() - t)

RuntimeError: ignored

Теперь нам нужно сложить в индекс все вопросы. Используем библиотеку annoy. Проходимся по всем ответам, считаем, что вектор предложения - сумма word2vecов слов, которые входят в него (конечно же усредненная)

In [ ]:
index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector = np.zeros(100)
        for word in question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)
            
        counter += 1

index.build(10)
index.save('speaker.ann')

Теперь остается реализовать метод, который получит на вход вопрос и найдет ответ к нему!
Мы препроцессим вопрос, находим ближайший вопрос и выбираем ответ на ближайший вопрос.

In [ ]:
def find_answer(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index.get_nns_by_vector(vector, 1)
    return index_map[answer_index[0]]